In [5]:
import numpy as np
import pandas as pd

## Análisis de datos sobre el Titanic

La fuente de datos es un csv sobre el Titanic y viene desde la [url] (https://raw.githubusercontent.com/datasciencedojo/datasets/refs/heads/master/titanic.csv)

>### Librerías

>### Carga de datos

In [6]:
titanic = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/refs/heads/master/titanic.csv')
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
df = titanic.copy() #hago una copia por si acaso durante el proceso, rompo la Base de datos 
           #y trabajo sobre df en vez de sobre Titanic DF

>### Diccionario de variables (features o columnas)

|Feature|Descripción| tipo de dato| clasificación de dato| tratamiento|
|---|---|---|---|---|
|``PasajeroID``|identificador único del pasajero|int|||
|``Survived``|si el pasajero sobrevivión, 0 y 1|int|binary/bool|
|``Edad``||||transformar a entero|
|``Cabina``||||eliminarla|

|Nombre|Descripción |tipo de dato |clasificación de dato |tratamiento |
|---|--- |---- |--- |--- |
|``PassengerId``|  identificador único del pasajero. |int |id |será indice |
|``Survived``| si el pasajero sobrevivió al naufragio, codificada como 0 (no) y 1 (si). |int |binary/bool | |
|``Pclass``| clase a la que pertenecía el pasajero: 1, 2 o 3. |str |categorical | |
|``Name``| nombre del pasajero |str |str | |
|``Sex``| sexo del pasajero. |str |binary/bool | |
|``Age``| edad del pasajero. |int |cuantitativo discreto | |
|``SibSp``|número de hermanos, hermanas, hermanastros o hermanastras en el barco. |str |cuantitativo discreto | |
|``Parch``|número de padres e hijos en el barco.|str |cuantitativo discreto| |
|``Ticket``|  identificador del billete. |str | | |
|``Fare``| precio pagado por el billete. |float|cuantitativo continuo | |
|``Cabin``| identificador del camarote asignado al pasajero. |object| |no interesa, eliminarla |
|``Embarked``| puerto en el que embarcó el pasajero. |str|cualitativo nominal | |

>### Análisis técnico


In [8]:
df.shape

(891, 12)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [10]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [11]:
(df.isnull().sum() / len(df)) * 100

PassengerId     0.000000
Survived        0.000000
Pclass          0.000000
Name            0.000000
Sex             0.000000
Age            19.865320
SibSp           0.000000
Parch           0.000000
Ticket          0.000000
Fare            0.000000
Cabin          77.104377
Embarked        0.224467
dtype: float64

*agv_dado que la fila Cabina tiene un 77% de desconocimiento, pues aunque yo impute valores, serán inventados, fantásticos, lo mejor será en esta caso, y con este conocimiento, eliminarla.
La edad tiene casí un 20%, con lo que intentaremos imputar esos datos, ya veremos cómo.* 

In [31]:
# df.unique() # Unique aplica solo a la Serie no al DagtaFrame
for col in df:
        print (f' porcentaje de únicos en la variable {col}, {round(len(df[col].unique())/len(df)*100,2)}, {len(df[col].unique())}')

 porcentaje de únicos en la variable PassengerId, 100.0, 891
 porcentaje de únicos en la variable Survived, 0.22, 2
 porcentaje de únicos en la variable Pclass, 0.34, 3
 porcentaje de únicos en la variable Name, 100.0, 891
 porcentaje de únicos en la variable Sex, 0.22, 2
 porcentaje de únicos en la variable Age, 8.08, 72
 porcentaje de únicos en la variable SibSp, 0.79, 7
 porcentaje de únicos en la variable Parch, 0.79, 7
 porcentaje de únicos en la variable Ticket, 76.43, 681
 porcentaje de únicos en la variable Fare, 27.83, 248
 porcentaje de únicos en la variable Cabin, 16.61, 148
 porcentaje de únicos en la variable Embarked, 0.45, 4


*agv_frecuencia de categorías...tendrá sentido hacerlo en aquellos casos en los que hay pocas categoría, pocas divisiones*

In [13]:
df['Survived'].value_counts(normalize=True)

Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64

In [14]:
#lo pongo más bonito y por categorías, mediante bucle
for col in ['Survived', 'Pclass', 'Sex','Embarked']:
    print(df[col].value_counts(normalize=True))
    print('------')
#en embarque

Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64
------
Pclass
3    0.551066
1    0.242424
2    0.206510
Name: proportion, dtype: float64
------
Sex
male      0.647587
female    0.352413
Name: proportion, dtype: float64
------
Embarked
S    0.724409
C    0.188976
Q    0.086614
Name: proportion, dtype: float64
------


In [15]:
#en embarque debe dar algo menos que 1, ya que hay algún None
print(df['Embarked'].value_counts(dropna().False))

SyntaxError: invalid syntax (1719623373.py, line 2)

In [ ]:
df

*agv_vamos a tratar el tema de la edad....*

In [ ]:
df['Age'].describe()

In [ ]:
df[df['Age'] <1]

In [ ]:
df[df['Age'] <1]['Age']

In [4]:
#se puede hacer con un lambda o con una mascara
#Juan prefiere este metodo 
df['Age'] =  df['Age'].apply(lambda row:1 if row<1 else row)

In [60]:
# con una mascara, pero no funciona, pues no puede comparar una columna completa con un valor
# df['Age'] = np.where (df['Age']< 1, 1, df['Age'])

In [ ]:
# df [df.loc [:,'Age']==1]['Age'] = 0

In [ ]:
# df[df['Age'] ==1]['Age'].replace (1,0)

In [ ]:
# df.loc [df['Age']<1. 'Age'] = 1

*agv_ ahora vamos a convertir los flotantes a enteros*


In [ ]:
df ['Age'] < 1

In [ ]:
df['Age'] = df ['Age'].apply (lambda row: int(row) if row != np.nan else row)

In [23]:
df['Age'] = df ['Age'].apply(lambda row: int (str(row).split ('.')[0]) if '.' in str(row) else row)

In [30]:
df['Age']

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888     NaN
889    26.0
890    32.0
Name: Age, Length: 891, dtype: float64

In [27]:
df['Age'] = df ['Age'].apply (lambda x: (x*10)//10) 


In [29]:
df['Age'].min()

np.float64(0.0)

In [34]:
df['Age'].astype('Int8')

0        22
1        38
2        26
3        35
4        35
       ... 
886      27
887      19
888    <NA>
889      26
890      32
Name: Age, Length: 891, dtype: Int8

_______________descanso